# Preprocesamiento para Experimentos de Actividad Evocada

Este Cuaderno es básicamente una versión interactiva del programa en el archivo "SeparaActividadyPromedia.jl" que contiene las mismas rutinas pero automatizadas. Esta versión permite un poco más de interacción.

In [1]:
include("SeparaActividadySaturados01.jl")
# en este modulo se encuentran las rutinas para encontrar los estimulos ("trancazos")
# y los canales saturados
using SeparaActividadySaturados01
using JLD

INFO: Recompiling stale cache file /home/karel/.julia/lib/v0.6/FileIO.ji for module FileIO.
INFO: Recompiling stale cache file /home/karel/.julia/lib/v0.6/JLD.ji for module JLD.


In [2]:
# Nombre de un archivo *brw en formato hdf5 con datos de acticidad evocada
abrestring="/home/karel/BRWFiles/Evocada05082015/050815_3R3_converted.brw"
stringgeneral=replace(abrestring, ".brw", "")  

"/home/karel/BRWFiles/Evocada05082015/050815_3R3_converted"

In [3]:
#Todos los datos de la matriz del brw file
Datos=AbreyCheca(abrestring)

Dict{String,Any} with 8 entries:
  "minvolt"     => -4125.0
  "maxvolt"     => 4125.0
  "frecuencia"  => 7020.8
  "DatosCrudos" => UInt16[0x0800 0x0000 … 0x0000 0x0000; 0x080b 0x0809 … 0x07f5…
  "bitdepth"    => 0x0c
  "factor"      => 2.01416
  "numcuadros"  => 210624
  "duracion"    => 30.0

In [4]:
# Si los datos andan en un arreglo de lista en lugar de cuadrado, los ponemos cuadrados
if size(Datos["DatosCrudos"])[1] != 4096
DatosCrudosArreglados=reshape(Datos["DatosCrudos"], (4096, Datos["numcuadros"]))
else
DatosCrudosArreglados=Datos["DatosCrudos"]
end;


In [5]:
#parametetros globales.
freq=Datos["frecuencia"]/1000 #cuadros por milisegundo.
factor=Datos["factor"] #Factor de conversion de numeros enteros a microVolts

retrazo=round(Int, ceil(5*freq))
final=round(Int, ceil(60*freq))
latencia=round(Int, ceil(1.5*freq))
#Gis recomienda esperar hasta 6.5 ms por cada estimulo.
tiempopostgolpe=round(Int,ceil(6.5*freq))


46

Tomamos un electrodo como ejemplo y buscamos los momentos (en cuadros, unidades enteras) de los estimulos:

In [6]:
ejemplo=vec(DatosCrudosArreglados[1500,:]);
listongas=EncuentraTrancazosRaw(ejemplo,500)
println("Sitios de trancazo electrico en cuadros =", listongas)

Sitios de trancazo electrico=[67863, 138471, 209079]


Ahora creamos un diccionario que contenga la actividad alrededor de los estimulos

In [7]:
ActividadRaw=ActivAlrededorTrancazo(listongas, DatosCrudosArreglados, freq)
(bla,tmax)=size(ActividadRaw["Trancazo_1"])
println("Tiempo, en cuadros, de los intervalos con trancazo=",tmax)


Tiempo, en cuadros, de los intervalos con trancazo=459


Finalmente, centramos y  reescalamos los datos, y los promediamos.

In [10]:
DatosCentrados=Dict{AbstractString, Array}()
for k in keys(ActividadRaw)
    DatosCentrados[k]=FormaMatrizDatosCentrados(ActividadRaw[k], factor)
end
LFPPromedio=(DatosCentrados["Trancazo_1"]+DatosCentrados["Trancazo_2"]+DatosCentrados["Trancazo_3"])/3;

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int32}, ::Int64, ::Int64, ::Int64) at ./deprecated.jl:57
 [3] FormaMatrizDatosCentrados(::Array{UInt16,2}, ::Float64) at /home/karel/RGutWork/Preprocesamiento/SeparaActividadySaturados01.jl:83
 [4] macro expansion at ./In[10]:3 [inlined]
 [5] anonymous at ./<missing>:?
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/karel/.julia/v0.6/IJulia/src/execute_request.jl:180
 [7] (::Compat.#inner#14{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/karel/.julia/v0.6/Compat/src/Compat.jl:332
 [8] eventloop(::ZMQ.Socket) at /home/karel/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##15#18)() at ./task.jl:335
while loading In[10], in expression starting on line 2


Eso fue la parte de promediar y poner escala sensible a los datos. Ahora prosigue detectar canales con respuesta fisiológica y canales saturados. Empecemos primero por las respuestas.

In [14]:
#los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, umbral en microvolts, umbral de saturación en microvolts
PruebaRespuesta=BuscaCanalRespActPot(LFPPromedio,freq, 120,-120,-800)
# y luego los saturados
iniciobusqueda=retrazo
finbusqueda=retrazo+4*latencia
Saturados=BuscaSaturados(LFPPromedio,1200,iniciobusqueda,finbusqueda)
numerosaturados=length(Saturados)
println("Encontramos ", numerosaturados, " canales probablemente saturados.")


Encontramos 229 canales probablemente saturados.


In [20]:
dirname(stringgeneral), basename(stringgeneral)

("/home/karel/BRWFiles/Evocada05082015", "050815_3R3_converted")

Finalmente podemos guardar los Datos

In [18]:
dirgen=dirname(stringgeneral)
basegen=basename(stringgeneral)
outname=string(,stringgeneral,".jld")
#aqui hay que arreglar el rollo del directorio
parametros=(freq, retrazo, latencia,final)
save(outname,
     "LFPSaturados", LFPPromedio,
     "freq",freq,
     "retrazo", retrazo,
     "latencia", latencia,
     "final", final,
     "Canalesrespuesta", PruebaRespuesta,
     "CanalesSaturados", Saturados)

HDF5-DIAG: Error detected in HDF5 (1.8.18) thread 0:
  #000: /var/tmp/portage/sci-libs/hdf5-1.8.18/work/hdf5-1.8.18/src/H5F.c line 522 in H5Fcreate(): unable to create file
    major: File accessibilty
    minor: Unable to open file
  #001: /var/tmp/portage/sci-libs/hdf5-1.8.18/work/hdf5-1.8.18/src/H5Fint.c line 1003 in H5F_open(): unable to open file: time = Mon Aug 27 13:46:26 2018
, name = 'LFP_Promedio_/home/karel/BRWFiles/Evocada05082015/050815_3R3_converted.jld', tent_flags = 13
    major: File accessibilty
    minor: Unable to open file
  #002: /var/tmp/portage/sci-libs/hdf5-1.8.18/work/hdf5-1.8.18/src/H5FD.c line 993 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #003: /var/tmp/portage/sci-libs/hdf5-1.8.18/work/hdf5-1.8.18/src/H5FDsec2.c line 339 in H5FD_sec2_open(): unable to open file: name = 'LFP_Promedio_/home/karel/BRWFiles/Evocada05082015/050815_3R3_converted.jld', errno = 2, error message = 'No such file or directory', 

LoadError: [91mError creating file LFP_Promedio_/home/karel/BRWFiles/Evocada05082015/050815_3R3_converted.jld[39m